## Plain logistic regression isn't looking promising

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# read functions.
import os
for f in os.listdir('fun/'): exec(open('fun/'+f).read())
del f

# Load data
load( 'out/d3-fight-level-standardize-normalize.pkl' )

# Change winner to binary 1/0:
y[ y == -1 ] = 0

If we just throw everything in, getting lots of bad p-values (and a pretty bad r-squared).

In [2]:
# Naive throw everything in
logit_model = sm.Logit( y, X )
result = logit_model.fit()
print(result.summary2())


Optimization terminated successfully.
         Current function value: 0.622251
         Iterations 7
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.054     
Dependent Variable:    Winner            AIC:               3037.7799 
Date:                  2019-11-17 20:53  BIC:               3491.7718 
No. Observations:      2314              Log-Likelihood:    -1439.9   
Df Model:              78                LL-Null:           -1522.0   
Df Residuals:          2235              LLR p-value:       4.1760e-08
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        7.0000                                         
----------------------------------------------------------------------
     Coef.      Std.Err.      z    P>|z|      [0.025         0.975]   
----------------------------------------------------------------------
x1    0.9762        0.6257  1.5602 0.1187        -0.2502        2.2026
x2 

c:\users\bryce\documents\python37\lib\site-packages\statsmodels\base\model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [3]:
# make predictions and check recall, precision, f1 score.

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score

pred = result.predict()
print( 
    'Mean wins: %s \nMean predict: %s\n' % ( 
    y.mean(),
    pred.mean()
))

# what is our base level if we predict the majority?
print( 'Accuracy predicting all wins:\n')
print( classification_report( 
    y, 
    [ 1 for x in pred ]
))

# what is the outcome of different cutoffs?
print( 'Accuracy with varying cutoffs:\n' )
for i in range(11): 
    
    icutoff = i/10
    
    predwin = [ 1 if x > i/10 else 0 for x in pred ]
    predloss = [ 0 if x > i/10 else 1 for x in pred ]
    
    fscorewin = f1_score( y, predwin )
    fscoreloss = f1_score( ( y == 0 ) * 1, predloss )    
    prec = precision_score( y, predwin )
    recall = recall_score( y, predwin )
    
    print(
        '%s: \t f1-score: %s   \t precision %s   \t recall: %s' % ( 
            i/10, 
            round( (fscorewin + fscoreloss) / 2, 2 ),
            round( prec, 2 ),
            round( recall, 2 )
    ))
    
print( '''
Seems like a cutoff of around .5 gives us way above average wins 
while participating in a large number of fights.
We are capturing 71% of the wins (recall) and winning 75% of the time (precision).
Strangely though, we could win 63% of the time and capture 100% of the wins by
always betting to win.
I guess we need to think about betting and what make the most sense.
Here are the stats for a .5 cutoff:
''')

print( classification_report( 
    y, 
    [ 1 if x > 0.5 else 0 for x in pred ]
))

Mean wins: 0.632238547968885 
Mean predict: 0.5479612642105314

Accuracy predicting all wins:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       851
           1       0.63      1.00      0.77      1463

    accuracy                           0.63      2314
   macro avg       0.32      0.50      0.39      2314
weighted avg       0.40      0.63      0.49      2314

Accuracy with varying cutoffs:

0.0: 	 f1-score: 0.39   	 precision 0.63   	 recall: 1.0
0.1: 	 f1-score: 0.39   	 precision 0.63   	 recall: 1.0
0.2: 	 f1-score: 0.43   	 precision 0.64   	 recall: 0.99
0.3: 	 f1-score: 0.53   	 precision 0.66   	 recall: 0.95
0.4: 	 f1-score: 0.61   	 precision 0.7   	 recall: 0.86
0.5: 	 f1-score: 0.65   	 precision 0.75   	 recall: 0.71
0.6: 	 f1-score: 0.63   	 precision 0.82   	 recall: 0.54
0.7: 	 f1-score: 0.51   	 precision 0.85   	 recall: 0.29
0.8: 	 f1-score: 0.38   	 precision 0.91   	 recall: 0.12
0.9: 	 f1-score: 0.28   	 p

c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Remove most of the insignificant features to see if something looks better. The only problem is that there isn't much predictive value regardless.

In [11]:
# prior model with incomplete dataset:

data = pd.read_csv("./out/d_fight_level_dataset_1line.csv", index_col = 0)

# Change winner to binary 1/0:\n",
data.Winner = data.Winner.apply(lambda x: np.where(x == -1, 0, 1))

# Initial features and target\n",
features = pd.Series(data.columns, index = data.columns)
target = "Winner"

# Remove referree, date, location, winner, title_bout, weight_class, no_of_rounds\n",
features.drop(index = ["Referee", "date", "location", "Winner", "title_bout",
                       "weight_class", "no_of_rounds"], inplace = True)

# Diff_draw is mostly NA/0\n",
features.drop(index = "Diff_draw", inplace = True)

# Lots of win columns\n",
features.drop(index = ["Diff_win_by_Decision_Majority",
                       "Diff_win_by_Decision_Split",
                       "Diff_win_by_Decision_Unanimous",
                       "Diff_win_by_KO/TKO",
                       "Diff_win_by_Submission",
                       "Diff_win_by_TKO_Doctor_Stoppage"], inplace = True)

# Naive throw everything in
logit_model = sm.Logit(data[target], data[features_adj])
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.622251
         Iterations 7
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.054     
Dependent Variable:    Winner            AIC:               3037.7799 
Date:                  2019-11-17 20:59  BIC:               3491.7718 
No. Observations:      2314              Log-Likelihood:    -1439.9   
Df Model:              78                LL-Null:           -1522.0   
Df Residuals:          2235              LLR p-value:       4.1760e-08
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        7.0000                                         
----------------------------------------------------------------------
     Coef.      Std.Err.      z    P>|z|      [0.025         0.975]   
----------------------------------------------------------------------
x1    0.9762        0.6257  1.5602 0.1187        -0.2502        2.2026
x2 

In [10]:
features_adj = features.drop(index = result.summary2().tables[1].index[result.summary2().tables[1]["P>|z|"] > .15])
logit_model = sm.Logit(data[target], data[features_adj])
result = logit_model.fit()
print(result.summary2())

KeyError: "['x2' 'x3' 'x7' 'x9' 'x10' 'x11' 'x12' 'x13' 'x14' 'x15' 'x16' 'x17' 'x19'\n 'x20' 'x21' 'x22' 'x23' 'x25' 'x26' 'x27' 'x28' 'x29' 'x30' 'x31' 'x32'\n 'x35' 'x36' 'x38' 'x40' 'x41' 'x42' 'x43' 'x44' 'x47' 'x50' 'x51' 'x52'\n 'x53' 'x54' 'x55' 'x56' 'x57' 'x58' 'x59'] not found in axis"

Maybe the effects are different by weight class? These results show more promise in some cases (though probably not enough effectiveness for a betting strategy).

In [ ]:
classes = pd.DataFrame(data.weight_class.value_counts())
classes.drop(index = classes.index[np.where(classes.weight_class < 100)], inplace = True)

for x in range(len(classes.index)):
    df = data.loc[data.weight_class == classes.index[x]]
    print("Class: " + classes.index[x])
    logit_model = sm.Logit(df[target], df[features_adj])
    result = logit_model.fit()
    print(result.summary2())